In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

C:\Users\OWNER\AppData\Local\Temp\ipykernel_12788\2213573283.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\OWNER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is alread

True

In [2]:
file_path = r'C:\Users\OWNER\Desktop\AnyoneAI\final_project\assignment\alpha2_dataset_cleaned.csv'

df = pd.read_csv(file_path)
df = df.fillna(pd.NA)

print(df.head())
df.info()

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [3]:
# stemmer, lemmatizer and stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from typing import Optional

# Initialize NLTK resources
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'  # Adjective
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun if not recognized

def remove_extra_new_lines(text):

    if pd.isnull(text):  # check if text is nan
        return ''  # replace with an empty string

    clean_text = [i for i in str(text).splitlines() if i.strip()]
    clean_text = ' '.join(clean_text)
    return clean_text

def remove_extra_whitespace(text: str) -> str:

    spaceless_text = re.sub(r'\s+', ' ', text)
    return spaceless_text

def remove_special_chars(text: str, remove_digits: Optional[bool] = False) -> str:

    if remove_digits:
        pattern = r'[^a-zA-Z\s]'
    else:
        pattern = r'[^a-zA-Z0-9\s]'

    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

def normalize_text(text):

    text = remove_extra_new_lines(text)

    text = remove_extra_whitespace(text)

    text = remove_special_chars(text, remove_digits=False)

    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    tagged_tokens = pos_tag(tokens)
    lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in tagged_tokens]

    return ' '.join(lemmas)

In [4]:
normalization = ['name', 'description']
for column in normalization:
    df[column + '_normalized'] = df[column].apply(normalize_text)

print(df.shape)

(50041, 13)


In [5]:
print(df.iloc[0])


name                                      Duracell - AAA Batteries (4-Pack)
type                                                               HardGood
price                                                                  5.49
description               Compatible with select electronic devices; AAA...
manufacturer                                                       Duracell
url                                           duracell aaa batteries 4 pack
parent_category                                 Connected Home & Housewares
sub_category_1                                                   Housewares
sub_category_2                                          Household Batteries
sub_category_3                                           Alkaline Batteries
sub_category_4                                                         <NA>
name_normalized                                        duracell aaa battery
description_normalized    compatible select electronic device aaa size d...
Name: 0, dty

In [6]:
#df['sub_category_1'].fillna('0', inplace=True)
#df['sub_category_2'].fillna('0', inplace=True)
#df['sub_category_3'].fillna('0', inplace=True)
#df['sub_category_4'].fillna('0', inplace=True)


In [7]:
print(df.head())
df.shape
X = df.drop(columns=['parent_category'])
y = df['parent_category']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [8]:
df.shape

X_1 = df.drop(columns=['sub_category_1'])
y_1 = df['sub_category_1']
y_1.fillna('missing', inplace=True)

label_encoder = LabelEncoder()
y_1_encoded = label_encoder.fit_transform(y_1)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1_encoded, test_size=0.2, random_state=42)

In [9]:
df.shape
X_2 = df.drop(columns=['sub_category_2'])
y_2 = df['sub_category_2']
y_2.fillna('missing', inplace=True)
label_encoder = LabelEncoder()
y_2_encoded = label_encoder.fit_transform(y_2)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2_encoded, test_size=0.2, random_state=42)

In [10]:
df.shape
X_3 = df.drop(columns=['sub_category_3'])
y_3 = df['sub_category_3']
y_3.fillna('missing', inplace=True)

label_encoder = LabelEncoder()
y_3_encoded = label_encoder.fit_transform(y_3)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_3, y_3_encoded, test_size=0.2, random_state=42)

In [11]:
df.shape
X_4 = df.drop(columns=['sub_category_4'])
y_4 = df['sub_category_4']
y_4.fillna('missing', inplace=True)

label_encoder = LabelEncoder()
y_4_encoded = label_encoder.fit_transform(y_4)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X_4, y_4_encoded, test_size=0.2, random_state=42)

In [12]:
print(X_4.head())


                                              name      type  price  \
0                Duracell - AAA Batteries (4-Pack)  HardGood   5.49   
1  Duracell - AA 1.5V CopperTop Batteries (4-Pack)  HardGood   5.49   
2                 Duracell - AA Batteries (8-Pack)  HardGood   7.49   
3            Energizer - MAX Batteries AA (4-Pack)  HardGood   4.99   
4                  Duracell - C Batteries (4-Pack)  HardGood   8.99   

                                         description manufacturer  \
0  Compatible with select electronic devices; AAA...     Duracell   
1  Long-lasting energy; DURALOCK Power Preserve t...     Duracell   
2  Compatible with select electronic devices; AA ...     Duracell   
3  4-pack AA alkaline batteries; battery tester i...    Energizer   
4  Compatible with select electronic devices; C s...     Duracell   

                                           url              parent_category  \
0                duracell aaa batteries 4 pack  Connected Home & Housewares   


In [13]:
print(X_train.shape)
print( X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_train['name_normalized'])

(40032, 12)
(10009, 12)
(40032,)
(10009,)
9233                            star fox preowned nintendo
25631    pioneer networkready ultra hd passthrough av h...
19030                     evolve ultimate edition xbox one
12044    joby pro series ultraplate quickrelease plate ...
18967    aluratek bump w home audio speaker system ipod...
                               ...                        
11284    samsung class diag lead curved smart ultra hd ...
44732    hifonics brutus class mono mosfet subwoofer am...
38158    mobile edge premium laptop backpack apple macb...
860                                 presonus presonus gray
15795      insignia portable bluetooth stereo speaker blue
Name: name_normalized, Length: 40032, dtype: object


In [14]:
y_train

array([20,  1, 20, ...,  6, 16,  1])

One Hot Encoder y Scaler


Stage 1

In [15]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Define columns
categorical_columns = ['type', 'manufacturer']
numerical_columns = ['price']
text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train[categorical_columns] = X_train[categorical_columns].fillna('missing')
X_test[categorical_columns] = X_test[categorical_columns].fillna('missing')

# One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train[categorical_columns])
X_test_encoded = encoder.transform(X_test[categorical_columns])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded:", X_train_encoded.shape)
print("Data type of X_train_encoded:", type(X_train_encoded))
print("Data type of elements in X_train_encoded:", X_train_encoded.dtype)
print("Shape of X_test_encoded:", X_test_encoded.shape)
print("Data type of X_test_encoded:", type(X_test_encoded))
print("Data type of elements in X_test_encoded:", X_test_encoded.dtype)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numerical_columns]) 
X_test_scaled = scaler.transform(X_test[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Data type of elements in X_train_scaled:", X_train_scaled.dtype)
print("Shape of X_test_scaled:", X_test_scaled.shape)
print("Data type of elements in X_test_scaled:", X_test_scaled.dtype)

Shape of X_train_encoded: (40032, 2195)
Data type of X_train_encoded: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded: float64
Shape of X_test_encoded: (10009, 2195)
Data type of X_test_encoded: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded: float64
Shape of X_train_scaled: (40032, 1)
Data type of elements in X_train_scaled: float64
Shape of X_test_scaled: (10009, 1)
Data type of elements in X_test_scaled: float64


In [67]:
from tensorflow.keras.models import load_model
from transformers import AutoTokenizer
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

                      
user_input = {
    'name': 'Sample Product',
    'description': 'This is a sample product description. It is used for demonstration purposes.',
    'price': 999.99,
    'type': 'Sample Type',
    'manufacturer': 'Sample Manufacturer',
}


model_1 = load_model('model_1_preberttune.h5')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')


def extract_last_hidden_state(embeddings):
    return embeddings[:, -1, :]
    
def tokenize_and_get_embeddings(column):
    # Tokenize text
    if isinstance(column, str):
        tokenized_text = tokenizer(column, padding=True, truncation=True, return_tensors='tf')
    elif isinstance(column, list) and all(isinstance(item, str) for item in column):
        tokenized_text = tokenizer(column, padding=True, truncation=True, return_tensors='tf')
    else:
        raise ValueError("Invalid input format.")
    
    outputs = model(tokenized_text)
    embeddings = outputs.last_hidden_state.numpy()
    
    return embeddings
    
def prepare_input(user_input, scaler, encoder, categorical_columns):

    user_input['name'] = normalize_text(user_input['name'])
    user_input['description'] = normalize_text(user_input['description'])

    name_embeddings = tokenize_and_get_embeddings(user_input['name'])
    description_embeddings = tokenize_and_get_embeddings(user_input['description'])
    extracted_name_hidden = extract_last_hidden_state(name_embeddings)
    extracted_description_hidden = extract_last_hidden_state(description_embeddings)
    scaled_price = scaler.transform([[user_input['price']]])[0, 0]

    encoded_user_input = [[user_input[column]] for column in categorical_columns]
    encoded_user_input = np.array(encoded_user_input).reshape(1, -1)
    encoded_categorical_features = encoder.transform(encoded_user_input)
    print(encoded_categorical_features.shape)

    combined_features = hstack([encoded_categorical_features, np.array([[scaled_price]])])
    print(combined_features.shape)
    num_cat_input_array = combined_features.toarray().astype(np.float32)
    final_input_array = np.concatenate((num_cat_input_array,extracted_name_hidden,extracted_description_hidden), axis=1)

    return name_embeddings, description_embeddings, final_input_array

name_embeddings, description_embeddings, final_input_array = prepare_input(user_input, scaler, encoder, categorical_columns)


categories_dict = {
    'parent_category': {category: index for index, category in enumerate(df['parent_category'].unique())},
    'sub_category_1': {category: index for index, category in enumerate(df['sub_category_1'].unique())},
    'sub_category_2': {category: index for index, category in enumerate(df['sub_category_2'].unique())},
    'sub_category_3': {category: index for index, category in enumerate(df['sub_category_3'].unique())},
    'sub_category_4': {category: index for index, category in enumerate(df['sub_category_4'].unique())},
}



def predict_model_1(final_input_array, model_1):
    predictions = model_1.predict(final_input_array)  # creo que este es el modelo correspondiente // acá cargo el h5 vector
    subcategory_pred_labels = np.argmax(predictions, axis=1)  
    return subcategory_pred_labels

def compare_predictions(subcategory_pred_labels, categories_dict, category_type):
    mapping = {v: k for k, v in categories_dict[category_type].items()}  
    predicted_labels = [mapping[idx] for idx in subcategory_pred_labels] 

    return predicted_labels

predicted_labels = predict_model_1(final_input_array, model_1)
predicted_labels_2 = compare_predictions(predicted_labels, categories_dict, 'parent_category')

#INCLUIR HASTA AQUI EN PIPELINE

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

(1, 2195)
(1, 2196)
1/1 [==============================] - 0s 62ms/step


C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [64]:
print(predicted_labels)

[19]


In [65]:
print(predicted_labels_2)

['H/VG_X360/Games/B2G1_20130602']


Stage 2

In [23]:
# Define columns
categorical_columns_1 = ['type', 'manufacturer', 'parent_category']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_1[categorical_columns_1] = X_train_1[categorical_columns_1].fillna('missing')
X_test_1[categorical_columns_1] = X_test_1[categorical_columns_1].fillna('missing')

# One-hot encode categorical features
encoder_1 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_1 = encoder_1.fit_transform(X_train_1[categorical_columns_1])
X_test_encoded_1 = encoder_1.transform(X_test_1[categorical_columns_1])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_1:", X_train_encoded_1.shape)
print("Data type of X_train_encoded_1:", type(X_train_encoded_1))
print("Data type of elements in X_train_encoded_1:", X_train_encoded_1.dtype)
print("Shape of X_test_encoded_1:", X_test_encoded_1.shape)
print("Data type of X_test_encoded_1:", type(X_test_encoded_1))
print("Data type of elements in X_test_encoded_1:", X_test_encoded_1.dtype)

# Scale numerical features
scaler_1 = StandardScaler()
X_train_scaled_1 = scaler_1.fit_transform(X_train_1[numerical_columns])
X_test_scaled_1 = scaler_1.transform(X_test_1[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_1:", X_train_scaled_1.shape)
print("Data type of elements in X_train_scaled_1:", X_train_scaled_1.dtype)
print("Shape of X_test_scaled_1:", X_test_scaled_1.shape)
print("Data type of elements in X_test_scaled_1:", X_test_scaled_1.dtype)

Shape of X_train_encoded_1: (40032, 2218)
Data type of X_train_encoded_1: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_1: float64
Shape of X_test_encoded_1: (10009, 2218)
Data type of X_test_encoded_1: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_1: float64
Shape of X_train_scaled_1: (40032, 1)
Data type of elements in X_train_scaled_1: float64
Shape of X_test_scaled_1: (10009, 1)
Data type of elements in X_test_scaled_1: float64


Stage 3

In [24]:
# Define columns
categorical_columns_2 = ['type', 'manufacturer', 'parent_category', 'sub_category_1']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_2[categorical_columns_2] = X_train_2[categorical_columns_2].fillna('missing')
X_test_2[categorical_columns_2] = X_test_2[categorical_columns_2].fillna('missing')

# One-hot encode categorical features
encoder_2 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_2 = encoder_2.fit_transform(X_train_2[categorical_columns_2])
X_test_encoded_2 = encoder_2.transform(X_test_2[categorical_columns_2])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_2:", X_train_encoded_2.shape)
print("Data type of X_train_encoded_2:", type(X_train_encoded_2))
print("Data type of elements in X_train_encoded_2:", X_train_encoded_2.dtype)
print("Shape of X_test_encoded_2:", X_test_encoded_2.shape)
print("Data type of X_test_encoded_2:", type(X_test_encoded_2))
print("Data type of elements in X_test_encoded_2:", X_test_encoded_2.dtype)

# Scale numerical features
scaler_2 = StandardScaler()
X_train_scaled_2 = scaler_2.fit_transform(X_train_2[numerical_columns])
X_test_scaled_2 = scaler_2.transform(X_test_2[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_2:", X_train_scaled_2.shape)
print("Data type of elements in X_train_scaled_2:", X_train_scaled_2.dtype)
print("Shape of X_test_scaled_2:", X_test_scaled_2.shape)
print("Data type of elements in X_test_scaled_2:", X_test_scaled_2.dtype)

Shape of X_train_encoded_2: (40032, 2332)
Data type of X_train_encoded_2: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_2: float64
Shape of X_test_encoded_2: (10009, 2332)
Data type of X_test_encoded_2: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_2: float64
Shape of X_train_scaled_2: (40032, 1)
Data type of elements in X_train_scaled_2: float64
Shape of X_test_scaled_2: (10009, 1)
Data type of elements in X_test_scaled_2: float64


Stage 4

In [25]:
# Define columns
categorical_columns_3 = ['type', 'manufacturer', 'parent_category', 'sub_category_1', 'sub_category_2']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_3[categorical_columns_3] = X_train_3[categorical_columns_3].fillna('missing')
X_test_3[categorical_columns_3] = X_test_3[categorical_columns_3].fillna('missing')

# One-hot encode categorical features
encoder_3 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_3 = encoder_3.fit_transform(X_train_3[categorical_columns_3])
X_test_encoded_3 = encoder_3.transform(X_test_3[categorical_columns_3])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_3:", X_train_encoded_3.shape)
print("Data type of X_train_encoded_3:", type(X_train_encoded_3))
print("Data type of elements in X_train_encoded_3:", X_train_encoded_3.dtype)
print("Shape of X_test_encoded_3:", X_test_encoded_3.shape)
print("Data type of X_test_encoded_3:", type(X_test_encoded_3))
print("Data type of elements in X_test_encoded_3:", X_test_encoded_3.dtype)

# Scale numerical features
scaler_3 = StandardScaler()
X_train_scaled_3 = scaler_3.fit_transform(X_train_3[numerical_columns])
X_test_scaled_3 = scaler_3.transform(X_test_3[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_3:", X_train_scaled_3.shape)
print("Data type of elements in X_train_scaled_3:", X_train_scaled_3.dtype)
print("Shape of X_test_scaled_3:", X_test_scaled_3.shape)
print("Data type of elements in X_test_scaled_3:", X_test_scaled_3.dtype)

Shape of X_train_encoded_3: (40032, 2681)
Data type of X_train_encoded_3: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_3: float64
Shape of X_test_encoded_3: (10009, 2681)
Data type of X_test_encoded_3: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_3: float64
Shape of X_train_scaled_3: (40032, 1)
Data type of elements in X_train_scaled_3: float64
Shape of X_test_scaled_3: (10009, 1)
Data type of elements in X_test_scaled_3: float64


Stage 5

In [26]:
# Define columns
categorical_columns_4 = ['type', 'manufacturer', 'parent_category', 'sub_category_1', 'sub_category_2', 'sub_category_3']
# numerical_columns = ['price']
# text_columns = ['name_normalized', 'description_normalized']

# Fill missing values
X_train_4[categorical_columns_4] = X_train_4[categorical_columns_4].fillna('missing')
X_test_4[categorical_columns_4] = X_test_4[categorical_columns_4].fillna('missing')

# One-hot encode categorical features
encoder_4 = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_4 = encoder_4.fit_transform(X_train_4[categorical_columns_4])
X_test_encoded_4 = encoder_4.transform(X_test_4[categorical_columns_4])

# Print information about X_train_encoded and X_test_encoded
print("Shape of X_train_encoded_4:", X_train_encoded_4.shape)
print("Data type of X_train_encoded_4:", type(X_train_encoded_4))
print("Data type of elements in X_train_encoded_4:", X_train_encoded_4.dtype)
print("Shape of X_test_encoded_4:", X_test_encoded_4.shape)
print("Data type of X_test_encoded_4:", type(X_test_encoded_4))
print("Data type of elements in X_test_encoded_4:", X_test_encoded_4.dtype)

# Scale numerical features
scaler_4 = StandardScaler()
X_train_scaled_4 = scaler_4.fit_transform(X_train_4[numerical_columns])
X_test_scaled_4 = scaler_4.transform(X_test_4[numerical_columns])

# Print information about X_train_scaled and X_test_scaled
print("Shape of X_train_scaled_4:", X_train_scaled_4.shape)
print("Data type of elements in X_train_scaled_4:", X_train_scaled_4.dtype)
print("Shape of X_test_scaled_4:", X_test_scaled_4.shape)
print("Data type of elements in X_test_scaled_4:", X_test_scaled_4.dtype)

Shape of X_train_encoded_4: (40032, 2997)
Data type of X_train_encoded_4: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_train_encoded_4: float64
Shape of X_test_encoded_4: (10009, 2997)
Data type of X_test_encoded_4: <class 'scipy.sparse._csr.csr_matrix'>
Data type of elements in X_test_encoded_4: float64
Shape of X_train_scaled_4: (40032, 1)
Data type of elements in X_train_scaled_4: float64
Shape of X_test_scaled_4: (10009, 1)
Data type of elements in X_test_scaled_4: float64


In [27]:
from scipy.sparse import hstack

# Stage 1
X_train_processed = hstack([X_train_encoded, X_train_scaled]).astype(np.float32).toarray()
X_test_processed = hstack([X_test_encoded, X_test_scaled]).astype(np.float32).toarray()

# Stage 2
X_train_processed_1 = hstack([X_train_encoded_1, X_train_scaled_1]).astype(np.float32).toarray()
X_test_processed_1 = hstack([X_test_encoded_1, X_test_scaled_1]).astype(np.float32).toarray()

# Stage 3
X_train_processed_2 = hstack([X_train_encoded_2, X_train_scaled_2]).astype(np.float32).toarray()
X_test_processed_2 = hstack([X_test_encoded_2, X_test_scaled_2]).astype(np.float32).toarray()

# Stage 4
X_train_processed_3 = hstack([X_train_encoded_3, X_train_scaled_3]).astype(np.float32).toarray()
X_test_processed_3 = hstack([X_test_encoded_3, X_test_scaled_3]).astype(np.float32).toarray()

# Stage 5
X_train_processed_4 = hstack([X_train_encoded_4, X_train_scaled_4]).astype(np.float32).toarray()
X_test_processed_4 = hstack([X_test_encoded_4, X_test_scaled_4]).astype(np.float32).toarray()



In [28]:
# Dim 1
print("Data type of X_train_processed:", X_train_processed.dtype)
print("Data type of X_test_processed:", X_test_processed.dtype)
print("X_train_processed shape:", X_train_processed.shape)
print("X_test_processed shape:", X_test_processed.shape)

Data type of X_train_processed: float32
Data type of X_test_processed: float32
X_train_processed shape: (40032, 2196)
X_test_processed shape: (10009, 2196)


In [29]:
# Dim 2
print("Data type of X_train_processed_1:", X_train_processed_1.dtype)
print("Data type of X_test_processed_1:", X_test_processed_1.dtype)
print("X_train_processed_1 shape:", X_train_processed_1.shape)
print("X_test_processed_1 shape:", X_test_processed_1.shape)

Data type of X_train_processed_1: float32
Data type of X_test_processed_1: float32
X_train_processed_1 shape: (40032, 2219)
X_test_processed_1 shape: (10009, 2219)


In [30]:
# Dim 3
print("Data type of X_train_processed_2:", X_train_processed_2.dtype)
print("Data type of X_test_processed_2:", X_test_processed_2.dtype)
print("X_train_processed_2 shape:", X_train_processed_2.shape)
print("X_test_processed_2 shape:", X_test_processed_2.shape)

Data type of X_train_processed_2: float32
Data type of X_test_processed_2: float32
X_train_processed_2 shape: (40032, 2333)
X_test_processed_2 shape: (10009, 2333)


In [31]:
# Dim 4
print("Data type of X_train_processed_3:", X_train_processed_3.dtype)
print("Data type of X_test_processed_3:",X_test_processed_3.dtype)
print("X_train_processed_3 shape:", X_train_processed_3.shape)
print("X_test_processed_3 shape:", X_test_processed_3.shape)

Data type of X_train_processed_3: float32
Data type of X_test_processed_3: float32
X_train_processed_3 shape: (40032, 2682)
X_test_processed_3 shape: (10009, 2682)


In [32]:
# Dim 5
print("Data type of X_train_processed_4:", X_train_processed_4.dtype)
print("Data type of X_test_processed_4:", X_test_processed_4.dtype)
print("X_train_processed_4 shape:", X_train_processed_4.shape)
print("X_test_processed_4 shape:", X_test_processed_4.shape)

Data type of X_train_processed_4: float32
Data type of X_test_processed_4: float32
X_train_processed_4 shape: (40032, 2998)
X_test_processed_4 shape: (10009, 2998)


In [33]:
import numpy as np

# Define file paths to save the arrays
file_paths = {
    'X_train_processed.npy': X_train_processed,
    'X_test_processed.npy': X_test_processed,
    'X_train_processed_1.npy': X_train_processed_1,
    'X_test_processed_1.npy': X_test_processed_1,
    'X_train_processed_2.npy': X_train_processed_2,
    'X_test_processed_2.npy': X_test_processed_2,
    'X_train_processed_3.npy': X_train_processed_3,
    'X_test_processed_3.npy': X_test_processed_3,
    'X_train_processed_4.npy': X_train_processed_4,
    'X_test_processed_4.npy': X_test_processed_4
}

# Save each array
for file_name, array in file_paths.items():
    np.save(file_name, array)


In [34]:
X_train_name_embeddings_loaded = np.load('X_train_name_embeddings.npy', allow_pickle= True)
X_train_description_embeddings_loaded = np.load('X_train_description_embeddings.npy',allow_pickle= True)
X_test_name_embeddings_loaded = np.load('X_test_name_embeddings.npy', allow_pickle= True)
X_test_description_embeddings_loaded = np.load('X_test_description_embeddings.npy', allow_pickle= True)

# Load the saved NumPy arrays
X_train_processed_loaded = np.load('X_train_processed.npy', allow_pickle=True)
X_test_processed_loaded = np.load('X_test_processed.npy', allow_pickle=True)
X_train_processed_1_loaded = np.load('X_train_processed_1.npy', allow_pickle=True)
X_test_processed_1_loaded = np.load('X_test_processed_1.npy', allow_pickle=True)
X_train_processed_2_loaded = np.load('X_train_processed_2.npy', allow_pickle=True)
X_test_processed_2_loaded = np.load('X_test_processed_2.npy', allow_pickle=True)
X_train_processed_3_loaded = np.load('X_train_processed_3.npy', allow_pickle=True)
X_test_processed_3_loaded = np.load('X_test_processed_3.npy', allow_pickle=True)
X_train_processed_4_loaded = np.load('X_train_processed_4.npy', allow_pickle=True)
X_test_processed_4_loaded = np.load('X_test_processed_4.npy', allow_pickle=True)



In [35]:
X_train_name_last_hidden = np.array([x[0][-1] for x in X_train_name_embeddings_loaded])
X_train_description_last_hidden = np.array([x[0][-1] for x in X_train_description_embeddings_loaded])
X_test_name_last_hidden = np.array([x[0][-1] for x in X_test_name_embeddings_loaded])
X_test_description_last_hidden = np.array([x[0][-1] for x in X_test_description_embeddings_loaded])

In [36]:
print("First element of X_train_name_last_hidden:", X_train_name_last_hidden[0])

First element of X_train_name_last_hidden: [ 6.20537519e-01 -1.57028392e-01 -4.39105332e-01  5.87245941e-01
 -4.92567480e-01 -6.75922573e-01  3.66373003e-01 -7.91122556e-01
  6.63730741e-01  4.76211309e-02  4.97472771e-02 -3.77824754e-01
 -5.29451743e-02 -3.52970883e-03 -7.33473897e-01 -2.74465412e-01
 -8.19092095e-02 -1.52533606e-01  7.26244077e-02 -4.86062840e-02
  3.07591200e-01 -1.32735014e-01  8.47112298e-01  2.39423364e-01
  1.34860486e-01  3.51377934e-01 -4.78440404e-01 -1.80202276e-02
 -2.26639926e-01 -3.52599382e-01 -6.21600211e-01 -2.67526209e-01
 -1.92636084e-02  3.99675339e-01  1.55551210e-01 -8.61436129e-02
  3.39647025e-01 -4.60110493e-02 -5.71235001e-01 -4.14561182e-01
 -3.34018916e-01 -1.61173679e-02 -1.93653673e-01  6.32311583e-01
  5.47741950e-02 -6.00057304e-01  6.22461319e-01  3.42181236e-01
 -2.25496083e-01  4.89310294e-01  2.93806463e-01  1.60992384e-01
 -7.31535181e-02  8.67539346e-02  5.76151237e-02  1.21880889e-01
  2.94119507e-01 -4.34998453e-01  1.54902250e-0

In [37]:
print("Shape of X_train_name_last_hidden:", X_train_name_last_hidden.shape)
print("Data type of X_train_name_last_hidden:", X_train_name_last_hidden.dtype)

print("Shape of X_train_description_last_hidden:", X_train_description_last_hidden.shape)
print("Data type of X_train_description_last_hidden:", X_train_description_last_hidden.dtype)

print("Shape of X_test_name_last_hidden:", X_test_name_last_hidden.shape)
print("Data type of X_test_name_last_hidden:", X_test_name_last_hidden.dtype)

print("Shape of X_test_description_last_hidden:", X_test_description_last_hidden.shape)
print("Data type of X_test_description_last_hidden:", X_test_description_last_hidden.dtype)

Shape of X_train_name_last_hidden: (40032, 768)
Data type of X_train_name_last_hidden: float32
Shape of X_train_description_last_hidden: (40032, 768)
Data type of X_train_description_last_hidden: float32
Shape of X_test_name_last_hidden: (10009, 768)
Data type of X_test_name_last_hidden: float32
Shape of X_test_description_last_hidden: (10009, 768)
Data type of X_test_description_last_hidden: float32


In [38]:
X_train_concatenated = np.concatenate((X_train_name_last_hidden, X_train_description_last_hidden), axis=1)
X_test_concatenated = np.concatenate((X_test_name_last_hidden, X_test_description_last_hidden), axis=1)

In [39]:
print("Shape of X_train_concatenated:", X_train_concatenated.shape)
print("Data type of X_train_concatenated:", X_train_concatenated.dtype)

print("Shape of X_test_concatenated:", X_test_concatenated.shape)
print("Data type of X_test_concatenated:", X_test_concatenated.dtype)

Shape of X_train_concatenated: (40032, 1536)
Data type of X_train_concatenated: float32
Shape of X_test_concatenated: (10009, 1536)
Data type of X_test_concatenated: float32


In [40]:
# Stage 1
X_train_combined = np.concatenate((X_train_processed_loaded, X_train_concatenated), axis=1)
X_test_combined = np.concatenate((X_test_processed_loaded, X_test_concatenated), axis=1)

# Stage 2
X_train_combined_1 = np.concatenate((X_train_processed_1_loaded, X_train_concatenated), axis=1)
X_test_combined_1 = np.concatenate((X_test_processed_1_loaded, X_test_concatenated), axis=1)

# Stage 3
X_train_combined_2 = np.concatenate((X_train_processed_2_loaded, X_train_concatenated), axis=1)
X_test_combined_2 = np.concatenate((X_test_processed_2_loaded, X_test_concatenated), axis=1)

# Stage 4
X_train_combined_3 = np.concatenate((X_train_processed_3_loaded, X_train_concatenated), axis=1)
X_test_combined_3 = np.concatenate((X_test_processed_3_loaded, X_test_concatenated), axis=1)

# Stage 5
X_train_combined_4 = np.concatenate((X_train_processed_4_loaded, X_train_concatenated), axis=1)
X_test_combined_4 = np.concatenate((X_test_processed_4_loaded, X_test_concatenated), axis=1)


In [41]:
#Define num_clases

num_classes = len(df['parent_category'].unique())
num_classes_1 = len(df['sub_category_1'].unique())
num_classes_2 = len(df['sub_category_2'].unique())
num_classes_3 = len(df['sub_category_3'].unique())
num_classes_4 = len(df['sub_category_4'].unique())


{'Connected Home & Housewares': 0, 'other': 1, 'Car Electronics & GPS': 2, 'In-Store Only': 3, 'Musical Instruments': 4, 'Toys': 5, 'Video Games': 6, 'Cameras & Camcorders': 7, 'Computers & Tablets': 8, 'Appliances': 9, 'Audio': 10, 'TV & Home Theater': 11, 'Health': 12, 'Name Brands': 13, 'Cell Phones': 14, 'Movies & Music': 15, 'Magnolia Home Theater': 16, 'Geek Squad': 17, 'Best Buy Gift Cards': 18, 'H/VG_X360/Games/B2G1_20130602': 19, 'MP Exclusives': 20, 'Wearable Technology': 21, 'Custom Parts': 22}
{' Housewares': 0, 'missing': 1, ' Car Installation Parts & Accessories': 2, ' Telephones & Communication': 3, ' Car Audio': 4, ' Recording Equipment': 5, ' Games & Drones': 6, ' Musical Instrument Accessories': 7, ' Microphones & Live Sound': 8, ' Keyboards': 9, ' DJ & Lighting Equipment': 10, ' Sheet Music & DVDs': 11, ' Wii': 12, ' Digital Cameras': 13, ' Xbox 360': 14, ' Camcorder Accessories': 15, ' Karaoke': 16, ' Computer Accessories & Peripherals': 17, ' Pre-Owned Games': 18, 

In [42]:
print(df['parent_category'].unique())

['Connected Home & Housewares' 'other' 'Car Electronics & GPS'
 'In-Store Only' 'Musical Instruments' 'Toys' 'Video Games'
 'Cameras & Camcorders' 'Computers & Tablets' 'Appliances' 'Audio'
 'TV & Home Theater' 'Health' 'Name Brands' 'Cell Phones' 'Movies & Music'
 'Magnolia Home Theater' 'Geek Squad' 'Best Buy Gift Cards'
 'H/VG_X360/Games/B2G1_20130602' 'MP Exclusives' 'Wearable Technology'
 'Custom Parts']


In [59]:
print(df['sub_category_1'].unique())

[' Housewares' 'missing' ' Car Installation Parts & Accessories'
 ' Telephones & Communication' ' Car Audio' ' Recording Equipment'
 ' Games & Drones' ' Musical Instrument Accessories'
 ' Microphones & Live Sound' ' Keyboards' ' DJ & Lighting Equipment'
 ' Sheet Music & DVDs' ' Wii' ' Digital Cameras' ' Xbox 360'
 ' Camcorder Accessories' ' Karaoke' ' Computer Accessories & Peripherals'
 ' Pre-Owned Games' ' Office Electronics' ' Marine & Powersports'
 ' Ranges' ' Headphones' ' Small Kitchen Appliances'
 ' Freezers & Ice Makers' ' TV Stands' ' Computer Cards & Components'
 ' Fitness & Beauty' ' Apple' ' Nintendo DS' ' PlayStation 3'
 ' TV & Home Theater Accessories' ' Digital Camera Accessories' ' Heating'
 ' Cell Phone Accessories' ' Irons' ' Scanners'
 ' Office & School Supplies' ' TVs' ' iPad & Tablet Accessories' 'other'
 ' Incase' ' Office Furniture & Storage' ' Home Audio' ' Memory Cards'
 ' Refrigerators' ' Car Safety & Convenience'
 ' iPod & MP3 Player Accessories' ' Microwaves

In [60]:
print(df['sub_category_2'].unique())

[' Household Batteries' 'missing' ' Car Audio Installation Parts'
 ' Telephone Accessories' ' Car Subwoofers & Enclosures'
 ' Sound Recording Software' ' TV' ' Keyboard Accessories'
 ' Live Sound Speakers' ' Audio Interfaces' 'other'
 ' Midi Keyboards & Controllers' ' Microphones & Accessories'
 ' DJ Equipment Accessories' ' Recording Furniture & Stands' ' Games'
 ' Sheet Music' ' Wii Games' ' Digital SLR Cameras' ' Xbox 360 Games'
 ' Action Camcorder Accessories' ' DJ Lighting & Effects'
 ' Karaoke Machines' ' Surge Protectors & Power' ' Car Amplifiers'
 ' Powersports Audio' ' Monitor & Screen Accessories' ' Cooktops & Ovens'
 ' All Headphones' ' Blenders & Juicers' ' Upright Freezers' ' DJ Mixers'
 ' Mounts & Furniture' ' Power Supplies' ' Personal Care & Beauty'
 ' Over-Ear & On-Ear Headphones' ' Accessories' ' Nintendo DS Games'
 ' GPUs/Video Graphics Cards' ' Cordless Telephones' ' PS3 Games'
 ' Remote Controls' ' Camera Lenses' ' Laptop Accessories'
 ' Cooling & Air Quality' ' Ce

In [61]:
print(df['sub_category_3'].unique())

[' Alkaline Batteries' 'missing' ' Specialty Batteries'
 ' Deck Installation Parts' ' Cordless Phone Batteries' ' Car Subwoofers'
 ' Movie & Character Toys' ' Speaker Monitors' ' Microphones'
 ' Recording & Music Stands' ' Puzzles & Cards' ' DSLR Body & Lens'
 ' Action Camcorder Mounts' ' Battery Backup (UPS)' ' 2-Channel Amps'
 ' Monitor & Video Cables' ' Cooktops' ' Blenders'
 ' Projector Mounts & Screens' 'other' ' Hair Care' ' Over-Ear Headphones'
 ' Shavers & Trimmers' ' PCI Express Graphics Cards' ' Expansion Handsets'
 ' Remote Control Accessories' ' TV Mounts' ' DSLR Lenses'
 ' Laptop Chargers & Adapters' ' Air Conditioners' ' iPhone Cases & Clips'
 ' Portable Chargers/Power Packs' ' Polarizing Filters' ' Irons'
 ' Sheetfed Scanners' ' Printer Ink' ' Covers & Keyboard Folios'
 ' Wireless & Multiroom Components' ' Back-Up Cameras'
 ' Composite Video Cables' ' Amp Installation Parts'
 ' Universal Remote Controls' ' Tea & Espresso' ' Electric Dryers'
 ' Gas Dryers' ' Toaster Ovens

In [62]:
print(df['sub_category_4'].unique())

['missing' ' Dash Installation Kits' ' Deck Harnesses'
 ' Antennas & Adapters' ' Music Memorabilia'
 ' More Pop Culture Merchandise' ' Dynamic' 'other'
 ' Handlebar/Seatpost Mounts' ' Single-Serve Blenders'
 ' Projector Screens' ' Hair Dryers' " Women's Shavers" ' Condenser'
 ' Prime Lenses' ' Hair & Nail Cutting Kits' ' Window Air Conditioners'
 ' Hair Straighteners' ' iPhone 6 Cases' ' Fuses & Capacitors'
 ' Interfaces & Converters' ' Amp Installation Kits' ' Tripod Mounts'
 ' Coffee Pod Racks & Storage' ' Replacement Parts & Accessories'
 ' Rice Cookers' ' Gas Grills' ' Cookware' ' Grill Accessories'
 ' Trimmers' ' Curling Irons' ' All TV Stands'
 ' Music & Audio Editing Software' ' Wall Oven & Microwave Combos'
 ' Double Ovens' ' Wired & USB Mice' ' Wired & USB Keyboards' ' Softboxes'
 ' Activity Tracker Bands' ' Gas Ranges' ' Wine Chillers'
 ' Lighting Accessories' ' Wall Chargers & Power Adapters'
 ' Electric Ranges' ' Hand Blenders' ' Electric Grills'
 ' Wireless & Bluetooth Mic

In [43]:
print(num_classes)
print(num_classes_1)
print(num_classes_2)
print(num_classes_3)
print(num_classes_4)

23
114
349
316
162


In [44]:
#Stage1
from tensorflow.keras import models, layers


from tensorflow.keras import callbacks
from tensorflow.keras.models import save_model
from sklearn.metrics import classification_report

# Define learning rate function
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)

# Compile the model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Model fit
run_1 = model.fit(X_train_combined, y_train, epochs=60, batch_size=32,
                  validation_data=(X_test_combined, y_test),
                  callbacks=[early_stopping, lr_scheduler_callback])

save_model(model, 'model_1_preberttune.h5')

y_pred_probabilities = model.predict(X_test_combined)
y_pred = np.argmax(y_pred_probabilities, axis=1)

loss, accuracy = model.evaluate(X_test_combined, y_test)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)





Epoch 1/60


1251/1251 [==============================] - 5s 3ms/step - loss: 0.9662 - accuracy: 0.7384 - val_loss: 0.4768 - val_accuracy: 0.8675 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.5709 - accuracy: 0.8462 - val_loss: 0.4098 - val_accuracy: 0.8879 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.4617 - accuracy: 0.8756 - val_loss: 0.3154 - val_accuracy: 0.9131 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.4038 - accuracy: 0.8922 - val_loss: 0.2644 - val_accuracy: 0.9324 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.3738 - accuracy: 0.9013 - val_loss: 0.2563 - val_accuracy: 0.9340 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.3317 - accuracy: 0.9112 - val_loss: 0.2372 - val_accuracy: 0.9394 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [=======

C:\Users\OWNER\AppData\Local\Temp\ipykernel_12788\2724651961.py:46: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, 'model_1_preberttune.h5')


313/313 [==============================] - 0s 1ms/step - loss: 0.1809 - accuracy: 0.9559
Accuracy: 0.9559396505355835
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1712
           1       0.92      0.96      0.94       715
           2       0.98      0.98      0.98        91
           3       0.95      0.97      0.96       649
           4       0.96      0.97      0.96       505
           5       0.97      0.97      0.97      1392
           6       0.94      0.95      0.94      1247
           7       0.93      0.93      0.93       890
           8       0.00      0.00      0.00         6
           9       0.84      1.00      0.91        16
          10       0.00      0.00      0.00         3
          11       0.94      0.93      0.94       245
          12       0.00      0.00      0.00         1
          13       1.00      1.00      1.00        15
          14       0.93      0.48      0.63     

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [45]:
#Stage2
# Define learning rate function
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)

# Create model
model_2 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_1.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_1, activation='softmax')
])

# Compile model
model_2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_2 = model_2.fit(X_train_combined_1, y_train_1, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_1, y_test_1),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_2, 'model_2_preberttune.h5')

# Evaluate model
loss_2, accuracy_2 = model_2.evaluate(X_test_combined_1, y_test_1)
print("Accuracy for Stage 2 model:", accuracy_2)

# Generate classification report
y_pred_probabilities_2 = model_2.predict(X_test_combined_1)
y_pred_2 = np.argmax(y_pred_probabilities_2, axis=1)
report_2 = classification_report(y_test_1, y_pred_2)
print("Classification Report for Stage 2 model:")
print(report_2)



Epoch 1/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.8290 - accuracy: 0.5915 - val_loss: 0.6525 - val_accuracy: 0.8276 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.8776 - accuracy: 0.7640 - val_loss: 0.4223 - val_accuracy: 0.8740 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.6973 - accuracy: 0.8022 - val_loss: 0.3494 - val_accuracy: 0.8952 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.6213 - accuracy: 0.8186 - val_loss: 0.3055 - val_accuracy: 0.9046 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.5692 - accuracy: 0.8301 - val_loss: 0.2887 - val_accuracy: 0.9150 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.5363 - accuracy: 0.8391 - val_loss: 0.2746 - val_accuracy: 0.9179 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [============

C:\Users\OWNER\AppData\Local\Temp\ipykernel_12788\4019872855.py:41: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model_2, 'model_2_preberttune.h5')


313/313 [==============================] - 0s 1ms/step - loss: 0.1933 - accuracy: 0.9453
Accuracy for Stage 2 model: 0.9453491568565369
313/313 [==============================] - 0s 1ms/step
Classification Report for Stage 2 model:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90        35
           1       0.89      0.74      0.81        23
           2       1.00      1.00      1.00         6
           3       0.96      1.00      0.98        54
           4       0.94      0.92      0.93       128
           5       0.87      1.00      0.93        13
           6       1.00      0.88      0.93         8
           7       0.00      0.00      0.00         2
           8       1.00      0.96      0.98        79
           9       1.00      1.00      1.00         9
          10       0.96      0.92      0.94       156
          11       0.86      0.52      0.65        71
          12       1.00      0.55      0.71        11
          1

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [46]:
# Stage 3

def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.85
    elif epoch < 19:
        return lr * 0.75
    else:
        return lr * 0.7

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)


model_3 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_2.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_2, activation='softmax')
])

# Compile model
model_3.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_3 = model_3.fit(X_train_combined_2, y_train_2, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_2, y_test_2),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_3, 'model_3_preberttune.h5')

# Evaluate model
loss_3, accuracy_3 = model_3.evaluate(X_test_combined_2, y_test_2)
print("Accuracy for Stage 3 model:", accuracy_3)

# Generate classification report
y_pred_probabilities_3 = model_3.predict(X_test_combined_2)
y_pred_3 = np.argmax(y_pred_probabilities_3, axis=1)
report_3 = classification_report(y_test_2, y_pred_3)
print("Classification Report for Stage 3 model:")
print(report_3)



Epoch 1/60
1251/1251 [==============================] - 6s 4ms/step - loss: 2.8146 - accuracy: 0.4691 - val_loss: 1.3406 - val_accuracy: 0.7048 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.4910 - accuracy: 0.6539 - val_loss: 0.8169 - val_accuracy: 0.7912 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.1543 - accuracy: 0.7088 - val_loss: 0.6357 - val_accuracy: 0.8221 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 4ms/step - loss: 1.0036 - accuracy: 0.7356 - val_loss: 0.5503 - val_accuracy: 0.8440 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.9057 - accuracy: 0.7569 - val_loss: 0.4833 - val_accuracy: 0.8627 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.8386 - accuracy: 0.7698 - val_loss: 0.4864 - val_accuracy: 0.8653 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [============

C:\Users\OWNER\AppData\Local\Temp\ipykernel_12788\3792186953.py:41: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model_3, 'model_3_preberttune.h5')


313/313 [==============================] - 0s 2ms/step - loss: 0.3415 - accuracy: 0.9061
Accuracy for Stage 3 model: 0.9060845375061035
313/313 [==============================] - 0s 1ms/step
Classification Report for Stage 3 model:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00        18
           2       0.79      1.00      0.88        70
           3       1.00      0.33      0.50         3
           4       1.00      0.86      0.92         7
           5       0.67      1.00      0.80         6
           6       1.00      1.00      1.00         8
           7       1.00      1.00      1.00        10
           8       0.84      0.97      0.90        39
           9       1.00      0.50      0.67         6
          10       0.60      1.00      0.75         3
          11       0.91      0.91      0.91        35
          12       0.88      0.94      0.91       101
          1

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [47]:
# Stage 4
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)

# Create model
model_4 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_3.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_3, activation='softmax')
])

# Compile model
model_4.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_4 = model_4.fit(X_train_combined_3, y_train_3, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_3, y_test_3),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_4, 'model_4_preberttune.h5')

# Evaluate model
loss_4, accuracy_4 = model_4.evaluate(X_test_combined_3, y_test_3)
print("Accuracy for Stage 4 model:", accuracy_4)

# Generate classification report
y_pred_probabilities_4 = model_4.predict(X_test_combined_3)
y_pred_4 = np.argmax(y_pred_probabilities_4, axis=1)
report_4 = classification_report(y_test_3, y_pred_4)
print("Classification Report for Stage 4 model:")
print(report_4)


Epoch 1/60
1251/1251 [==============================] - 6s 4ms/step - loss: 2.5734 - accuracy: 0.5470 - val_loss: 1.3015 - val_accuracy: 0.7166 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.3942 - accuracy: 0.6906 - val_loss: 0.8040 - val_accuracy: 0.7996 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.0790 - accuracy: 0.7383 - val_loss: 0.6155 - val_accuracy: 0.8409 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 5s 4ms/step - loss: 0.9129 - accuracy: 0.7668 - val_loss: 0.4903 - val_accuracy: 0.8507 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 5s 4ms/step - loss: 0.8244 - accuracy: 0.7842 - val_loss: 0.4852 - val_accuracy: 0.8624 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 5s 4ms/step - loss: 0.7471 - accuracy: 0.7982 - val_loss: 0.4259 - val_accuracy: 0.8775 - lr: 6.3241e-04
Epoch 7/60
1251/1251 [============

C:\Users\OWNER\AppData\Local\Temp\ipykernel_12788\1281415934.py:40: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model_4, 'model_4_preberttune.h5')


313/313 [==============================] - 0s 1ms/step - loss: 0.2969 - accuracy: 0.9151
Accuracy for Stage 4 model: 0.9150764346122742
313/313 [==============================] - 0s 1ms/step
Classification Report for Stage 4 model:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       1.00      1.00      1.00        56
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         3
           4       0.42      0.62      0.50         8
           5       0.75      0.50      0.60         6
           6       0.50      0.50      0.50         4
           7       0.87      0.87      0.87        15
           8       0.00      0.00      0.00         4
           9       1.00      0.33      0.50         3
          10       0.00      0.00      0.00         1
          11       0.66      1.00      0.79        23
          12       1.00      1.00      1.00        22
          1

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [48]:
# Stage 5
print("Running Stage 5 model...")
def lr_scheduler(epoch, lr):
    if epoch < 4:
        return lr * 0.94
    elif epoch < 8:
        return lr * 0.9
    elif epoch < 16: 
        return lr * 0.80
    elif epoch < 19:
        return lr * 0.70
    else:
        return lr * 0.6

# Define common callbacks
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler_callback = callbacks.LearningRateScheduler(lr_scheduler)
# Create model
model_5 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_combined_4.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes_4, activation='softmax')
])

# Compile model
model_5.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit model
history_5 = model_5.fit(X_train_combined_4, y_train_4, epochs=60, batch_size=32,
                        validation_data=(X_test_combined_4, y_test_4),
                        callbacks=[early_stopping, lr_scheduler_callback])

# Save model
save_model(model_5, 'model_5_preberttune.h5')

# Evaluate model
loss_5, accuracy_5 = model_5.evaluate(X_test_combined_4, y_test_4)
print("Accuracy for Stage 5 model:", accuracy_5)

# Generate classification report
y_pred_probabilities_5 = model_5.predict(X_test_combined_4)
y_pred_5 = np.argmax(y_pred_probabilities_5, axis=1)
report_5 = classification_report(y_test_4, y_pred_5)
print("Classification Report for Stage 5 model:")
print(report_5)


Running Stage 5 model...
Epoch 1/60
1251/1251 [==============================] - 5s 4ms/step - loss: 1.5513 - accuracy: 0.7560 - val_loss: 0.6146 - val_accuracy: 0.8753 - lr: 9.4000e-04
Epoch 2/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.6798 - accuracy: 0.8590 - val_loss: 0.4545 - val_accuracy: 0.8861 - lr: 8.8360e-04
Epoch 3/60
1251/1251 [==============================] - 4s 4ms/step - loss: 0.5438 - accuracy: 0.8747 - val_loss: 0.3272 - val_accuracy: 0.9101 - lr: 8.3058e-04
Epoch 4/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.4635 - accuracy: 0.8846 - val_loss: 0.2750 - val_accuracy: 0.9254 - lr: 7.8075e-04
Epoch 5/60
1251/1251 [==============================] - 4s 3ms/step - loss: 0.4108 - accuracy: 0.8932 - val_loss: 0.2406 - val_accuracy: 0.9251 - lr: 7.0267e-04
Epoch 6/60
1251/1251 [==============================] - 5s 4ms/step - loss: 0.3656 - accuracy: 0.9000 - val_loss: 0.2381 - val_accuracy: 0.9328 - lr: 6.3241e-04
Epoch 7/6

C:\Users\OWNER\AppData\Local\Temp\ipykernel_12788\646761526.py:40: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model_5, 'model_5_preberttune.h5')


313/313 [==============================] - 0s 1ms/step - loss: 0.1707 - accuracy: 0.9493
Accuracy for Stage 5 model: 0.949345588684082
313/313 [==============================] - 0s 1ms/step
Classification Report for Stage 5 model:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.94      0.94      0.94        18
           2       0.00      0.00      0.00         2
           3       1.00      0.50      0.67         4
           4       1.00      0.80      0.89         5
           5       0.78      1.00      0.88        71
           6       1.00      0.60      0.75         5
           7       1.00      1.00      1.00        18
           8       0.00      0.00      0.00         4
           9       1.00      1.00      1.00         1
          10       0.93      0.78      0.85        32
          11       0.00      0.00      0.00         3
          12       0.75      1.00      0.86         6
          13

C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\OWNER\Desktop\AnyoneAI\final_project\venv2\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [49]:
model_1 = load_model('model_1_preberttune.h5')
model_2 = load_model('model_2_preberttune.h5')


user_input = np.array([[name, description, price, type, manufacturer]]) 
#Se le tiene que aplicar las funciones de normalizacion al input del usuario texto(name,desccription), usar scale para precio y one hot para categoricas.
#usar label encoder para target category
target_1 = [parent_category]

#Preprocessed user input
#                                                          Usar las categorias directo del df?
#Main category pred
main_category_pred_probs = model_1.predict(user_input)

# get labels for main category
main_category_pred_labels = np.argmax(main_category_pred_probs, axis=1) #Houseware [0.9,0.8,0.5,0.4]----> Houseware,0

#Concat user input with label  and preprocess to enter second model
#se tiene que aplicar one hot al output del primer modelo y usarlo como feature en el segundo, 
#se puede usar el output preprocesado del primero y solo unir con el one hot
user_input_with_pred = np.concatenate((user_input, main_category_pred_labels.reshape(-1, 1)), axis=1) 

#Predict subcategory using model_2 with the concatenated input
subcategory_pred_probs = model_2.predict(user_input_with_pred) #vector

# Getting index
subcategory_pred_labels = np.argmax(subcategory_pred_probs, axis=1) # indice del valor mas alto

# Print the subcategory prediction labels
print(subcategory_pred_labels)

NameError: name 'load_model' is not defined